In [1]:
import torch
import torch.utils.data as data
import torchnet as tnt
import numpy as np
import pandas as pd
import datetime as dt

from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from torch import Tensor
import os
import json
import pickle as pkl
import argparse
import pprint

from models.stclassifier import PseTae
from dataset import PixelSetData, PixelSetData_preloaded
from learning.focal_loss import FocalLoss
from learning.weight_init import weight_init
from learning.metrics import mIou, confusion_matrix_analysis

def parse(date):
    d = str(date)
    return int(d[:4]), int(d[4:6]), int(d[6:])


def interval_days(date1, date2):
    return abs((dt.datetime(*parse(date1)) - dt.datetime(*parse(date2))).days)


def date_positions(dates):
    pos = []
    for d in dates:
        pos.append(interval_days(d, dates[0]))
    return pos

/home/mhbokaei/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# folder = "/home/mhbokaei/shakouri/test/Satellite/dataset_folder100/"
# folder = "/home/mhbokaei/shakouri/test/Satellite/dataset_folder10000/"
# folder = "/home/mhbokaei/shakouri/test/Satellite/dataset_folder90000/"
# folder = "/home/mhbokaei/shakouri/SatelliteImage_TimeSeries_Classification/dataset_folder"

folder = "/home/mhbokaei/shakouri/test/Satellite/dataset_folder10000/"
data_folder = os.path.join(folder, 'DATA')
meta_folder = os.path.join(folder, 'META')
labels='label_19class'
npixel = 64
sub_classes=None
#######################################################################################
l = [f for f in os.listdir(data_folder) if f.endswith('.npy')]
#print("l:", l)
pid = [int(f.split('.')[0]) for f in l]
#print("1:",pid, "type:", type(pid))
pid = list(np.sort(pid))
#print("2:",pid, "type:", type(pid))
pid = list(map(str, pid))
#print("3:",pid, "type:", type(pid))
len0 = len(pid) ###er###
print("len0:", len0)

######################################################################################
print("sub_classes:", sub_classes)
if sub_classes is not None:
  sub_indices = []
  num_classes = len(sub_classes)
  convert = dict((c, i) for i, c in enumerate(sub_classes))
#print("convert:", convert)
#print("num_classes:", num_classes)

with open(os.path.join(folder, 'META', 'labels.json'), 'r') as file:
  d = json.loads(file.read())
  target = []
  ii = []  ###er###
  pp = []  ###er###
  for i, p in enumerate(pid):
    t = d[labels][p]
    target.append(t)
    ii.append(i)  ###er###
    pp.append(p)  ###er###
    if sub_classes is not None:
      if t in sub_classes:
        sub_indices.append(i)
        target[-1] = convert[target[-1]]  ##change real class number to new with convert

#print("file:", file)
#print("ii:", ii)
#print("pp:", pp)
#print("target:", target)
#print("sub_indices:", sub_indices)
if sub_classes is not None:
  pid = list(np.array(pid)[sub_indices])
  target = list(np.array(target)[sub_indices])
  len1 = len(sub_indices)  ###er###
#print("pid:", pid)
#print("target:", target)
#print("len1:", len1)

##########################################################################################
##Date###
with open(os.path.join(folder, 'META', 'dates.json'), 'r') as file:
  d = json.loads(file.read())

dates = [d[str(i)] for i in range(len(d))]
date_positions = date_positions(dates)


#print("d:", d)
#print("dates:", dates)
#print("date_positions:", date_positions)

#####extra_feature#####
extra_feature = 'geomfeat'
if extra_feature is not None:
  with open(os.path.join(meta_folder, '{}.json'.format(extra_feature)), 'r') as file:
    extra = json.loads(file.read())
  if isinstance(extra[list(extra.keys())[0]], int):
    for k in extra.keys():
      extra[k] = [extra[k]]
  df = pd.DataFrame(extra).transpose()
  extra_m, extra_s = np.array(df.mean(axis=0)), np.array(df.std(axis=0))

#print("df:", df)
#print("extra_m:", extra_m)
#print("extra_s:", extra_s)


len0: 9975
sub_classes: None


In [ ]:
for item in range(len0):
  print("pid[item]: ", pid[item])
  npixel = 64
  x0 = np.load(os.path.join(folder, 'DATA', '{}.npy'.format(pid[item])))  ##it returns Sample
  if x0.ndim != 3:
    print("Bpixel: ", pid[item], ", x0.shape: ", x0.shape, ", x0.ndim: ", x0.ndim)
  if x0.shape[0] != 24 or x0.shape[1] != 10:
    print("Bpixel: ", pid[item], ", x0.shape[0]: ", x0.shape[0], ", x0.shape[1]: ", x0.shape[1])


  

In [ ]:
#data = np.load("/home/mhbokaei/shakouri/SatelliteImage_TimeSeries_Classification/dataset_folder/DATA/170979.npy")
#print(data)

In [ ]:
Xqqq = np.array([[11, 22], [33, 44], [21, 12], [15, 18], [11, 22], [33, 44], [21, 12], [15, 18], [21, 12], [15, 18]])

len(Xqqq)

In [ ]:
def __len__(self):
    return self.len

In [ ]:
kf = KFold(n_splits=5, shuffle=False)
indices_seqqq = list(kf.split(list(range(len(dt)))))